In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import joblib

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import train_test_split
from warnings import simplefilter
simplefilter("ignore")

In [5]:
dataset_dir = Path("dataset")

## CONFIG 

In [3]:
store_nbr_list = ["1","2"]
family_list = ["AUTOMATIVE", "SEAFOOD"]
val_ratio = 0.2

In [50]:
train = pd.read_csv(dataset_dir / 'train.csv',                            
    usecols=['store_nbr', 'family', 'date', 'sales', 'onpromotion'],
    dtype={
        'store_nbr': 'category',
        'family': 'category',
        'sales': 'float32',
        'onpromotion': 'uint32'
    },
    parse_dates=['date'],
    infer_datetime_format=True,
)
train['date'] = train.date.dt.to_period('D')

train = (train
       .set_index(['store_nbr','family','date'])         # Setting MultiIndex to make unique identifiers for each 'sales' item
       .sort_index()
      )
# train = train[train.index.isin(store_nbr_list)]
train

sales  onpromotion
store_nbr family     date                              
1         AUTOMOTIVE 2013-01-01   0.000000            0
                     2013-01-02   2.000000            0
                     2013-01-03   3.000000            0
                     2013-01-04   3.000000            0
                     2013-01-05   5.000000            0
...                                    ...          ...
9         SEAFOOD    2017-08-11  23.830999            0
                     2017-08-12  16.859001            4
                     2017-08-13  20.000000            0
                     2017-08-14  17.000000            0
                     2017-08-15  16.000000            0

[3000888 rows x 2 columns]

In [53]:
c1=train.index.get_level_values(0).isin(['1', '2', '3'])
c2 = train.index.get_level_values(1).isin(["AUTOMOTIVE","SEAFOOD"])
c3 = train.index.get_level_values(2).isin([])
train = train[c1 & c2]


sales  onpromotion
store_nbr family     date                              
1         AUTOMOTIVE 2013-01-01   0.000000            0
                     2013-01-02   2.000000            0
                     2013-01-03   3.000000            0
                     2013-01-04   3.000000            0
                     2013-01-05   5.000000            0
...                                    ...          ...
3         SEAFOOD    2017-08-11  94.312996            7
                     2017-08-12  71.917000            0
                     2017-08-13  77.398003            0
                     2017-08-14  86.451996            0
                     2017-08-15  71.060997            0

[10104 rows x 2 columns]

In [60]:

start_date = "2013-01-02"
end_date = "2013-01-06"
filtered_df = train[(train.index.get_level_values('date') >= start_date) & (train.index.get_level_values('date') <= end_date)]
filtered_df.to_csv('filtered_df.csv')

In [37]:
train=train.xs((["1","2"],"AUTOMOTIVE"),level=("store_nbr","family"))
train

KeyError: ('1', '2')

In [43]:
m1 = train.index.get_level_values(0).isin(["1","2"])
m2 = train.index.get_level_values(1).isin(["AUTOMOTIVE"])
train = train[m1 | m2]
train.index

MultiIndex([('1', 'AUTOMOTIVE', '2013-01-01'),
            ('1', 'AUTOMOTIVE', '2013-01-02'),
            ('1', 'AUTOMOTIVE', '2013-01-03'),
            ('1', 'AUTOMOTIVE', '2013-01-04'),
            ('1', 'AUTOMOTIVE', '2013-01-05'),
            ('1', 'AUTOMOTIVE', '2013-01-06'),
            ('1', 'AUTOMOTIVE', '2013-01-07'),
            ('1', 'AUTOMOTIVE', '2013-01-08'),
            ('1', 'AUTOMOTIVE', '2013-01-09'),
            ('1', 'AUTOMOTIVE', '2013-01-10'),
            ...
            ('2',    'SEAFOOD', '2017-08-06'),
            ('2',    'SEAFOOD', '2017-08-07'),
            ('2',    'SEAFOOD', '2017-08-08'),
            ('2',    'SEAFOOD', '2017-08-09'),
            ('2',    'SEAFOOD', '2017-08-10'),
            ('2',    'SEAFOOD', '2017-08-11'),
            ('2',    'SEAFOOD', '2017-08-12'),
            ('2',    'SEAFOOD', '2017-08-13'),
            ('2',    'SEAFOOD', '2017-08-14'),
            ('2',    'SEAFOOD', '2017-08-15')],
           names=['store_nbr', 'family', 'd

In [23]:
train = train[np.in1d(train.index.get_level_values(0)==["1","2"])]
train

InvalidIndexError: (array([ True,  True,  True, ..., False, False, False]), array([False, False, False, ..., False, False, False]))

,,,sales,onpromotion
store_nbr,family,date,,


In [13]:
train = train[np.in1d(train.index.get_level_values(0),["1"])]
train

,,,sales,onpromotion
store_nbr,family,date,,


In [70]:
test = pd.read_csv(dataset_dir/'test.csv',
    dtype={
        'store_nbr': 'category',
        'family': 'category',
        'onpromotion': 'uint32',
    },
    parse_dates=['date'],
    infer_datetime_format=True,
)
test['date'] = test.date.dt.to_period('D')
test = test.set_index(['store_nbr','family','date']).sort_index()
test = test[test.index.isin(store_nbr_list)]
test

id  onpromotion
store_nbr family     date                            
1         AUTOMOTIVE 2017-08-16  3000888            0
                     2017-08-17  3002670            0
                     2017-08-18  3004452            0
                     2017-08-19  3006234            0
                     2017-08-20  3008016            0
...                                  ...          ...
2         SEAFOOD    2017-08-27  3020885            1
                     2017-08-28  3022667            0
                     2017-08-29  3024449            1
                     2017-08-30  3026231            1
                     2017-08-31  3028013            0

[1056 rows x 2 columns]

In [96]:
df_test = pd.DataFrame([])
for store_nbr in store_nbr_list:
    df = test[test.index.isin([store_nbr])]
    

                                      id  onpromotion
store_nbr family     date                            
1         AUTOMOTIVE 2017-08-16  3000888            0
                     2017-08-17  3002670            0
                     2017-08-18  3004452            0
                     2017-08-19  3006234            0
                     2017-08-20  3008016            0
...                                  ...          ...
          SEAFOOD    2017-08-27  3020522            0
                     2017-08-28  3022304            1
                     2017-08-29  3024086            0
                     2017-08-30  3025868            1
                     2017-08-31  3027650            0

[528 rows x 2 columns]
                                      id  onpromotion
store_nbr family     date                            
2         AUTOMOTIVE 2017-08-16  3001251            0
                     2017-08-17  3003033            0
                     2017-08-18  3004815            0
    

In [ ]:
def split():
    

In [ ]:
class MODELS:
    def __init__(model_name):
        pass
        

In [75]:
from sktime.forecasting.exp_smoothing import ExponentialSmoothing
from sktime.forecasting.sarimax import SARIMAX
from sktime.forecasting.arima import AutoARIMA
from sktime.split import temporal_train_test_split
from sktime.forecasting.base import ForecastingHorizon

In [86]:
fh = ForecastingHorizon(
    pd.PeriodIndex(pd.date_range('2017-08-16', periods=15, freq="D")), is_relative=False
)
fh

ForecastingHorizon(['2017-08-16', '2017-08-17', '2017-08-18', '2017-08-19',
             '2017-08-20', '2017-08-21', '2017-08-22', '2017-08-23',
             '2017-08-24', '2017-08-25', '2017-08-26', '2017-08-27',
             '2017-08-28', '2017-08-29', '2017-08-30'],
            dtype='period[D]', is_relative=False)

In [88]:
fh2 = ForecastingHorizon(np.arange(1,16))
fh2

ForecastingHorizon([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], dtype='int32', is_relative=True)

In [90]:
forecaster =  ExponentialSmoothing(trend="add", seasonal="additive", sp=12)
forecaster.fit(train)

C:\Users\takyol\Desktop\kaggle_projects\store_sales\.venv\lib\site-packages\statsmodels\tsa\holtwinters\model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
C:\Users\takyol\Desktop\kaggle_projects\store_sales\.venv\lib\site-packages\statsmodels\tsa\holtwinters\model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
C:\Users\takyol\Desktop\kaggle_projects\store_sales\.venv\lib\site-packages\statsmodels\tsa\holtwinters\model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
C:\Users\takyol\Desktop\kaggle_projects\store_sales\.venv\lib\site-packages\statsmodels\tsa\holtwinters\model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
C:\Users\takyol\Desktop\kaggle_projects\store_sales\.venv\lib\site-packages\statsmodels\tsa\holtwinters\model.py:918: ConvergenceWarning: Optimization failed to converge. C

ExponentialSmoothing(seasonal='additive', sp=12, trend='add')

In [91]:
y_pred = forecaster.predict(fh)

In [92]:
y_pred

sales  onpromotion
store_nbr family     date                              
1         AUTOMOTIVE 2017-08-16   4.184816    -0.008264
                     2017-08-17   4.548707    -0.001126
                     2017-08-18   4.776941     0.006004
                     2017-08-19   4.811494    -0.001111
                     2017-08-20   4.825580    -0.008236
...                                    ...          ...
2         SEAFOOD    2017-08-26  35.416426     1.009510
                     2017-08-27  35.851301     1.066283
                     2017-08-28  34.986016     0.975691
                     2017-08-29  35.131448     1.161207
                     2017-08-30  33.237516     0.924279

[990 rows x 2 columns]

In [37]:
y_pred

family     AUTOMOTIVE                                                   \
family     AUTOMOTIVE                                                    
store_nbr           1        10       11        12        13        14   
date                                                                     
2017-08-25   4.483274  1.811184  7.89436  5.932794  5.003217  3.829323   

family                                              ...   SEAFOOD             \
family                                              ...   SEAFOOD              
store_nbr         15        16        17        18  ...         5         50   
date                                                ...                        
2017-08-25  3.946422  4.902763  9.191286  4.312973  ...  9.746943  19.963582   

family                                                                     \
family                                                                      
store_nbr          51        52        53        54          6          7   
date                                                                        
2017-08-25  50.982105  9.403427  4.284481  2.579434  37.288262  43.194023   

family                            
family                            
store_nbr           8          9  
date                              
2017-08-25  35.799047  17.768955  

[1 rows x 1782 columns]

MultiIndex([('AUTOMOTIVE', 'AUTOMOTIVE',  '1'),
            ('AUTOMOTIVE', 'AUTOMOTIVE', '10'),
            ('AUTOMOTIVE', 'AUTOMOTIVE', '11'),
            ('AUTOMOTIVE', 'AUTOMOTIVE', '12'),
            ('AUTOMOTIVE', 'AUTOMOTIVE', '13'),
            ('AUTOMOTIVE', 'AUTOMOTIVE', '14'),
            ('AUTOMOTIVE', 'AUTOMOTIVE', '15'),
            ('AUTOMOTIVE', 'AUTOMOTIVE', '16'),
            ('AUTOMOTIVE', 'AUTOMOTIVE', '17'),
            ('AUTOMOTIVE', 'AUTOMOTIVE', '18'),
            ...
            (   'SEAFOOD',    'SEAFOOD',  '5'),
            (   'SEAFOOD',    'SEAFOOD', '50'),
            (   'SEAFOOD',    'SEAFOOD', '51'),
            (   'SEAFOOD',    'SEAFOOD', '52'),
            (   'SEAFOOD',    'SEAFOOD', '53'),
            (   'SEAFOOD',    'SEAFOOD', '54'),
            (   'SEAFOOD',    'SEAFOOD',  '6'),
            (   'SEAFOOD',    'SEAFOOD',  '7'),
            (   'SEAFOOD',    'SEAFOOD',  '8'),
            (   'SEAFOOD',    'SEAFOOD',  '9')],
           names=['fami